In [1]:
import scipy

import numpy as np

import pandas as pd
from pandas import Series, DataFrame

from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint

import csv


import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly
plotly.tools.set_credentials_file(username='ra123', api_key='JtjRYEowQy5bMB6Z9lUD')

from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

# getting around the ascii characters
from django.utils.encoding import smart_str, smart_unicode

In [2]:
# Import the clean tweets (text and date, then through machine learning)
twitter_raw = pd.read_csv("C:\\Users\\Student24\\Documents\\twitter\\useing\\Clean_Tweets1.csv", sep=',', delimiter=None)

# Delete tweets containing key words

In [3]:
twitter_cleaned=twitter_raw[twitter_raw['text'].str.lower().str.contains \
                   ("great smog of london|spanish|sex|porn|anal|pov|bbw|milf|sexy|shemale|sexyfishrestaurant|\
                   nude|sluts|super hot blonde|adult video|erotic|18+|dirty fun|killer fog|rihanna")==False]

twitter_cleaned = twitter_cleaned.reset_index(drop=True)

twitter_cleaned['text'].count()

8940

#  Machine Learning to clean tweets

# 1. Create the Maching learning

In [4]:
twlist = []

with open(r"C:\\Users\\Student24\\Documents\\twitter\\trained_weather_NoRT.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

tweets = tweets_raw[['text', 'weather']].values.tolist()

twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [5]:
for (words, weather) in twlist:
#    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    words_filtered = [unicode(e.lower(), errors = 'replace') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [6]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [7]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [8]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [9]:
training_set = nltk.classify.apply_features(extract_features, tweets)

# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

print classifier.show_most_informative_features(32)

Most Informative Features
    contains(summer���s) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
      contains(@pc24fym) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
        contains(1012mb) = False              no : yes    =      1.0 : 1.0
       contains(provide) = False              no : yes    =      1.0 : 1.0
    contains(@moosawi17) = False              no : yes    =      1.0 : 1.0
      contains(mentally) = False              no : yes    =      1.0 : 1.0
          contains(#now) = False              no : yes    =      1.0 : 1.0
         contains(rainy) = False              no : yes    =      1.0 : 1.0

# 2. Classify London Tweets

In [10]:
df = pd.DataFrame(twitter_cleaned)

tx = df['text']
df['text'] = tx

In [11]:
dffinal = df[['text']]
dffinal['date'] = df['Date']

In [12]:
for i in range(len(df.index)):
    if classifier.classify(extract_features((smart_str(df['text'][i])).split())) == 'yes':
        dffinal['text'][i] = smart_str(df['text'][i])
    else:
        dffinal['text'][i] = None

count = 0
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        count += 1

count2 = 0
dffinaltrained = pd.DataFrame({'date' : pd.Series(range(count), index=range(count)), 'text' : pd.Series(range(count), index=range(count))})
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        dffinaltrained['text'][count2] = dffinal['text'][i]
        dffinaltrained['date'][count2] = dffinal['date'][i]
        count2 += 1

C:\Users\Student24\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Student24\Anaconda2\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [13]:
dffinaltrained['text'].count()

8093

# Define function which will count the weather words

In [14]:
def wordCount(tweet):
    # List of words we are looking at
    weather_words = ['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet', 'heat', 'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 'warm', 'warmer',
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'hot', 'sun', 'sunny',
                     'boiling', 'baltic', 'burn']
    # Create a new dictionnary
    counts = dict()
    for t in tweet:
        if t in weather_words:
            counts[t] = counts.get(t,0) + 1

    return counts

# Get the number of final clean tweets per day

In [15]:
# Check the days and number of tweets per day
dffinaltrained.groupby(['date']).count()

,text
date,
01/01/2017,431
02/01/2017,272
14/12/2016,300
15/12/2016,255
16/12/2016,795
19/12/2016,791
20/12/2016,804
21/12/2016,421
22/12/2016,463


In [16]:
# Create a new dataframe with the date
tw_date = dffinaltrained.groupby(['date']).count()
tw_date.index

Index([u'01/01/2017', u'02/01/2017', u'14/12/2016', u'15/12/2016',
       u'16/12/2016', u'19/12/2016', u'20/12/2016', u'21/12/2016',
       u'22/12/2016', u'23/12/2016', u'24/12/2016', u'25/12/2016',
       u'26/12/2016', u'27/12/2016', u'28/12/2016', u'30/12/2016',
       u'31/12/2016'],
      dtype='object', name=u'date')

In [17]:
tw_date['text'][1]

272

# Going through all tweets then count words and put them in final DataFrame (per day)

In [18]:
# Initialise the final dataframe
tw_final = DataFrame()
tw_final['date'] = tw_date.index

tw_final['Keyword_1'] = None
tw_final['Keyword_2'] = None
tw_final['Keyword_3'] = None
tw_final['Keyword_4'] = None
tw_final['Keyword_5'] = None

In [19]:
for i in range(len(tw_date.index)):
    df = dffinaltrained[(dffinaltrained['date'] == tw_final['date'][i])]
    twitterlist = []
    for t in range(len(df.index)):
        twitterlist.append(dffinaltrained['text'][t].lower().split())

    # Flatten the list so all values in the same list
    tweets_flatten = [j for sublist in twitterlist for j in sublist]

    # Counting the number of key words in the tweets
    d = wordCount(tweets_flatten)
    
    # give a list of the sorted tweets, with most popular coming first
    d2 = sorted(d.items(), key = lambda x: x[1], reverse = True)

    
    tw_final['Keyword_1'][i] = d2[0]#[0]  #keyword 1 is the most popular
    tw_final['Keyword_2'][i] = d2[1]#[0]  #keyword 2 is the second most popular
    tw_final['Keyword_3'][i] = d2[2]#[0]
    tw_final['Keyword_4'][i] = d2[3]#[0]
    tw_final['Keyword_5'][i] = d2[4]#[0]

In [20]:
tw_final

,date,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5
0,01/01/2017,"(hot, 49)","(sun, 44)","(fog, 43)","(cold, 41)","(warm, 28)"
1,02/01/2017,"(fog, 34)","(sun, 31)","(hot, 29)","(cold, 29)","(warm, 16)"
2,14/12/2016,"(fog, 35)","(hot, 33)","(cold, 33)","(sun, 32)","(warm, 17)"
3,15/12/2016,"(fog, 31)","(hot, 29)","(sun, 28)","(cold, 28)","(warm, 16)"
4,16/12/2016,"(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)"
5,19/12/2016,"(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)"
6,20/12/2016,"(hot, 76)","(snow, 66)","(cold, 63)","(fog, 61)","(sun, 60)"
7,21/12/2016,"(hot, 47)","(sun, 42)","(fog, 42)","(cold, 41)","(warm, 28)"
8,22/12/2016,"(hot, 55)","(fog, 49)","(sun, 45)","(cold, 43)","(warm, 32)"
9,23/12/2016,"(hot, 50)","(fog, 45)","(sun, 44)","(cold, 41)","(warm, 29)"


# Pivot and Join

In [21]:
# Unpivots tw_final
tw_final2 = pd.melt(tw_final, id_vars = ['date'])

In [22]:
tw_final2.head()

,date,variable,value
0,01/01/2017,Keyword_1,"(hot, 49)"
1,02/01/2017,Keyword_1,"(fog, 34)"
2,14/12/2016,Keyword_1,"(fog, 35)"
3,15/12/2016,Keyword_1,"(fog, 31)"
4,16/12/2016,Keyword_1,"(hot, 74)"


In [23]:
tw_final2.rename(columns = {'variable':'rank'}, inplace = True)

In [24]:
# Seperates tuples in value
b = []
c = []

b.extend(a[1] for a in tw_final2['value'])
tw_final2['count'] = b

c.extend(a[0] for a in tw_final2['value'])
tw_final2['value'] = c

In [25]:
tw_final2.head()

,date,rank,value,count
0,01/01/2017,Keyword_1,hot,49
1,02/01/2017,Keyword_1,fog,34
2,14/12/2016,Keyword_1,fog,35
3,15/12/2016,Keyword_1,fog,31
4,16/12/2016,Keyword_1,hot,74


In [26]:
# Creates dataframe of total tweets for each day
df_count = dffinaltrained.groupby(['date']).count()

In [27]:
# Resets index
df_count.reset_index(level = 0, inplace = True)

In [28]:
df_count.rename(columns = {'text':'total_count'}, inplace = True)

In [29]:
df_count.head()

,date,total_count
0,01/01/2017,431
1,02/01/2017,272
2,14/12/2016,300
3,15/12/2016,255
4,16/12/2016,795


In [30]:
# Joins dataframes tw_final2 and df_count on date
merged = pd.merge(tw_final2, df_count, how='inner', on='date')

In [31]:
# Creates column for %daily word occurrence
merged['%daily word occurrence'] = (merged['count'] / merged['total_count']) * 100

In [32]:
merged.rename(columns = {'value':'key_word'}, inplace = True)

In [33]:
merged.head()

,date,rank,key_word,count,total_count,%daily word occurrence
0,01/01/2017,Keyword_1,hot,49,431,11.368910
1,01/01/2017,Keyword_2,sun,44,431,10.208817
2,01/01/2017,Keyword_3,fog,43,431,9.976798
3,01/01/2017,Keyword_4,cold,41,431,9.512761
4,01/01/2017,Keyword_5,warm,28,431,6.496520


In [34]:
merged.to_csv(path_or_buf="C:\\Users\\Student24\\Documents\\twitter\\useing\\Daily_Word_Occurrence.csv", sep=',', na_rep='', \
               float_format=None, columns=['date', 'rank', 'key_word', 'count', 'total_count', '%daily word occurrence'], header=True, index=False, index_label=None, mode='w', encoding=None, \
               compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, \
               date_format=None, doublequote=True, escapechar=None, decimal='.')
# Change header = false and mode = a from header = true and mode = w

# Date vs Count |Table+Plot

In [82]:
merged_piv=merged.pivot(index='date',columns='key_word', values='count')

In [93]:
merged_piv.reset_index(level = 0, inplace = True)

In [146]:
merged_piv

key_word,date,cold,fog,hot,snow,sun,warm
0,01/01/2017,41.0,43.0,49.0,NaN,44.0,28.0
1,02/01/2017,29.0,34.0,29.0,NaN,31.0,16.0
2,14/12/2016,33.0,35.0,33.0,NaN,32.0,17.0
3,15/12/2016,28.0,31.0,29.0,NaN,28.0,16.0
4,16/12/2016,60.0,61.0,74.0,66.0,60.0,NaN
5,19/12/2016,60.0,61.0,74.0,66.0,60.0,NaN
6,20/12/2016,63.0,61.0,76.0,66.0,60.0,NaN
7,21/12/2016,41.0,42.0,47.0,NaN,42.0,28.0
8,22/12/2016,43.0,49.0,55.0,NaN,45.0,32.0
9,23/12/2016,41.0,45.0,50.0,NaN,44.0,29.0


In [188]:
# convert the df to a numpy array
np_counts = np.array(merged_piv)
print np_counts.shape

(17L, 7L)


In [194]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace=[]
col_num = len(merged_piv.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num):
    if i==0:
        x_date=np_counts[:,i] # Sets the x-axis as the date
    else:
        y_count=np_counts[:,i].astype(float) #Sets y-axis as count
        trace.append(Scatter( x=x_date, y=y_count, name=merged_piv.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data = Data(trace) #list trace is not data for plotly plot
layout = go.Layout( #layout of plotly plot
    title='Occurrence of keywords for top 5 keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Count',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Occurrence_of_all_keywords') #plotly plot

u'https://plot.ly/~ra123/8'

# Date vs %_of_Count |Table+Plot

In [195]:
merged_piv_2=merged.pivot(index='date',columns='key_word', values='%daily word occurrence')
merged_piv_2.reset_index(level = 0, inplace = True)
merged_piv_2

key_word,date,cold,fog,hot,snow,sun,warm
0,01/01/2017,9.512761,9.976798,11.368910,NaN,10.208817,6.496520
1,02/01/2017,10.661765,12.500000,10.661765,NaN,11.397059,5.882353
2,14/12/2016,11.000000,11.666667,11.000000,NaN,10.666667,5.666667
3,15/12/2016,10.980392,12.156863,11.372549,NaN,10.980392,6.274510
4,16/12/2016,7.547170,7.672956,9.308176,8.301887,7.547170,NaN
5,19/12/2016,7.585335,7.711757,9.355247,8.343869,7.585335,NaN
6,20/12/2016,7.835821,7.587065,9.452736,8.208955,7.462687,NaN
7,21/12/2016,9.738717,9.976247,11.163895,NaN,9.976247,6.650831
8,22/12/2016,9.287257,10.583153,11.879050,NaN,9.719222,6.911447
9,23/12/2016,9.360731,10.273973,11.415525,NaN,10.045662,6.621005


In [196]:
# convert the df to a numpy array
np_counts_2 = np.array(merged_piv_2)
print np_counts_2.shape

(17L, 7L)


In [198]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace_2=[]
col_num_2 = len(merged_piv_2.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num_2):
    if i==0:
        x_date_2=np_counts_2[:,i] # Sets the x-axis as the date
    else:
        y_count_2=np_counts_2[:,i].astype(float) #Sets y-axis as count
        trace_2.append(Scatter( x=x_date_2, y=y_count_2, name=merged_piv_2.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data_2 = Data(trace_2) #list trace is not data for plotly plot
layout_2 = go.Layout( #layout of plotly plot
    title='% of keywords for top 5 keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='%',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig_2 = go.Figure(data=data_2, layout=layout_2)
py.plot(fig_2, filename = '%_of_all_keywords') #plotly plot

u'https://plot.ly/~ra123/10'